# Gestures

Demonstrates how small units of code can serve as building blocks for creating larger musical structures. Composer: Todd Ingalls.

Running this demo requires a jupyter kernel (runtime environment) that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px;color:gray">

Python imports:

In [ ]:
import random
from musx import version, setmidiplayer, playfile, Score, Seq, Note, MidiFile, Shuffle, odds, between, quantize, interp
from musx.midi.gm import AcousticGrandPiano, Marimba, OrchestralHarp
print(f'musx version:', version)

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")

Define the `motive1()` composer:

In [ ]:
def motive1(score, octave, maxtransp, chan):
    """
    Motive1 generates three notes in random order but always sounding a
    whole step and minor seventh. The motive can be randomly
    transposed within range half-steps.

    Parameters
    ----------
    score : Score
        The score.
    octave : int
        The octave to play the notes in.
    maxtransp : int
        The maximum transposition in half steps.
    chan : int
        The midi channel to assign to the notes.
    """
    # the basic pitches to transpose and suffle e.g. [F#4 E4 D5].
    pitches = Shuffle([6, 4, 14])
    # one of the three pitches will be louder than the others.
    amps = Shuffle([.75, .5, .5])
    # randomly chosen transpostion within a limited range
    offset = random.randrange(maxtransp)
    for _ in range(3):
        knum = pitches.next() + (octave * 12) + offset
        note = Note(time=score.now, duration=.1, pitch=knum, amplitude=amps.next(), instrument=chan)
        score.add(note)
        yield .2
        
print(f"motive1: {motive1}")

Preview `motive1()` several times:

In [ ]:
s = Score(out=Seq())
procs = [jazz-high-hat(120, 1) for ]
s.compose( [[t, motive1(s, 5, 8, 0)] for t in range(0, 10, 2)] )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing motive1()...")

`Motive2()` generates a repeated note with one of the pair always accented:

In [ ]:
def motive2(score, octave, maxtransp, chan):
    """Motive2 generates a repeated tone with one tone accented."""
    amps = Shuffle([.75, .5, .5])
    rhys = Shuffle([.2, .2, .4])
    offset = random.randrange(maxtransp)
    for _ in range(3):
        knum = 0 + (octave * 12) + offset
        note = Note(time=score.now, duration=.1, pitch=knum, amplitude=amps.next(), instrument=chan)
        score.add(note)
        yield rhys.next()
print(f"motive2: {motive2}")

Preview `motive2()` several times:

In [ ]:
s = Score(out=Seq())
s.compose( [[t, motive2(s, 5, 5, 1)] for t in range(0, 10, 2)] )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing motive2()...")

`gesture1()` chooses between `motive1()` and `motive2()` with a two second rest in between:

In [ ]:
def gesture1(score, numtimes, o, chan):
    for _ in range(numtimes):
        if (odds(o)):
            score.compose(motive1(score, 5, 1, chan))
        else:
            score.compose(motive2(score, 6, 1, chan))
        yield 2
print(f"gesture1: {gesture1}")

Preview `gesture1()` several times:

In [ ]:
s = Score(out=Seq())
s.compose( gesture1(s, 10, .5, 0) )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing gesture1()...")

`gesture2()` is the same as `gesture1()` but supports transpositions:

In [ ]:
def gesture2(score, numtimes, o, maxtransp, chan):
    for _ in range(numtimes):
        if (odds(o)):
            score.compose(motive1(score, 5, maxtransp, chan))
        else:
            score.compose(motive2(score, 6, maxtransp, chan))
        yield 2
print(f"gesture2: {gesture2}")

Preview `gesture2()` several times:

In [ ]:
s = Score(out=Seq())
s.compose( gesture2(s, 10, .5, 5, 0) )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing gesture2()...")

Define a function `qtime()` that quantizes rhythms:

In [ ]:
def qtime(n, total, start, end, quant):
    """
    Over total time move from start to end by quant step size. The
    end value is reached half-way through and sticks thereafter.
    """
    return quantize(interp(n / total, 0, start, .5, end), quant)
print(f"qtime: {qtime}")

Test it out:

In [ ]:
for i in range(10):
  print(qtime(i, 10, 2, 4, .25))

Like `gesture2()` but uses `qtime()` to speed up.

In [ ]:
def gesture3(score, numtimes, o, limit, chan, hiwait, lowwait):
    for i in range(numtimes):
        if (odds(o)):
            score.compose(motive1(score, 5, limit, chan))
        else:
            score.compose(motive2(score, 6, limit, chan))
        yield qtime(i, numtimes, 2, .2, .2)
print(f"gesture3: {gesture3}")

Preview `gesture3()`:

In [ ]:
s = Score(out=Seq())
s.compose( gesture3(s, 20, .5, 5, 0, 3, .2) )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing gesture3()...")

`gesture4()` is similar to `gesture3()` but chooses octaves and gradually prefers motive2 over motive1:

In [ ]:
def gesture4(score, numtimes, lowoctave, highoctave, limit, chan, hiwait, lowwait):
    for i in range(numtimes):
        if odds(qtime(i, numtimes, 1.0, 0.0, .01)):
            score.compose(motive1(score, between(lowoctave, highoctave), limit, chan))
        else:
            score.compose(motive2(score, between(lowoctave, highoctave), limit, chan))
        yield qtime(i, numtimes, hiwait, lowwait, .2)
print(f"gesture4: {gesture4}")

Preview `gesture4()`:

In [ ]:
s = Score(out=Seq())
s.compose( gesture4(s, 30, 2, 7, 11, 0, 1.6,.2) )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing gesture4()...")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: AcousticGrandPiano, 1: Marimba, 2: OrchestralHarp})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to receive the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Create a composition using three variations of `gesture4()` to be performed by three different instruments: Piano, Marimba, and Harp: 

In [ ]:
score = Score(out=track1)

trio = [gesture4(score, 60, 2, 7, 11, 0, 1.0, .2),  # piano
        gesture4(score, 40, 5, 7, 11, 1, 1.6, .2),  # Marimba
        gesture4(score, 34, 3, 6, 11, 2, 2.0, .2)]  # Harp
score.compose( trio )
print("OK!")

Write the tracks to a midi file in the current directory:

In [ ]:
file =  MidiFile("gestures.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if a terminal based midi player is installed:

In [ ]:
playfile(file.pathname)